# CSR、SSR、SSG、ISR

本文主要介绍四种常见的前端渲染方式。文中涉及到 React 和 Next.js 默认指的是版本 18 和 14。

## CSR（Client-side Rendering 客户端渲染）

客户端渲染指的是网页的渲染过程发生在浏览器。

浏览器从服务器请求一个 HTML 文件，这个文件通常包含一个用于挂载 JS 应用的根 DOM 元素，例如：

```html
<div id="root"></div>
```

React 通过 API 将应用挂载到这个根 DOM 元素上，从而渲染出完整的页面内容。调用过程如下：

```tsx
import React from "react";
import ReactDOM from "react-dom/client";

ReactDOM.createRoot(document.getElementById("root")!).render(<App />);
```

Babel 将 JSX 语法转换为对 React 元素构造函数的调用。比如：

```tsx
<div>hello, world</div>
```

将会被转换为：

```js
import { jsx as _jsx } from "react/jsx-runtime";
/*#__PURE__*/ _jsx("div", {
  children: "hello, world",
});
```

React 通过 `_jsx` 方法创建 React 元素，并对这些元素进行处理，构建出实际的 DOM 结构。

最后，这些 DOM 会被挂载到 #root 根 DOM 上，从而实现页面渲染。

客户端渲染适合高动态单页应用（SPA），例如后台管理系统。

由于这些应用通常需要频繁的与用户交互、动态更新页面内容，通过在客户端进行渲染，可减少页面重新加载的次数，快速响应用户操作，从而提升整体的交互性能。

### 优势：

**增强用户体验**

客户端渲染将页面的渲染和更新放在前端进行，避免了服务器再次生成完整页面造成的整体页面刷新，因此可以实现即时的交互反馈和页面的部分更新。

**服务器负担小**

数据处理和页面渲染在客户端进行，减少了服务器的计算和渲染压力，在高并发场景下效果尤为显著。

### 不足：

**首屏加载慢**

由于页面内容的渲染是在客户端进行的，因此页面所有的 JS 代码块（chunks）都需要从服务器传输到浏览器执行。
较大的 JS 文件会导致加载时间延长，用户可能会看到短暂的白屏或闪烁现象。

**SEO 问题**

传统的搜索引擎爬虫主要依赖静态 HTML 内容进行索引。然而，客户端渲染的内容是通过 JS 动态生成的，这会对 SEO 产生影响。

根据 Vercel 团队的最新测试结果，现代搜索引擎（如 Google）能够有效处理和索引 JS 渲染的内容。参考 [How Google handles JavaScript throughout the indexing process](https://vercel.com/blog/how-google-handles-javascript-throughout-the-indexing-process#myth-1-%E2%80%9Cgoogle-can%E2%80%99t-render-javascript-content%E2%80%9D)。

然而，Baidu（百度）在处理 JS 渲染内容方面的支持仍然有限，且目前没有明确的计划进行技术升级。相关信息请参考 [百度搜索引擎工作原理](https://ziyuan.baidu.com/college/courseinfo?id=144)。

拓展：

- [了解 Google JavaScript SEO](https://developers.google.com/search/docs/crawling-indexing/javascript/javascript-seo-basics?hl=zh-cn)。

- [Babel 在线转换 JSX](https://babeljs.io/repl#?browsers=defaults%2C%20not%20ie%2011%2C%20not%20ie_mob%2011&build=&builtIns=false&corejs=3.21&spec=false&loose=false&code_lz=DwEwlgbgfAFgpgGwQewDQAIDuyBOCTAD040QA&debug=false&forceAllTransforms=false&modules=false&shippedProposals=false&evaluate=false&fileSize=false&timeTravel=false&sourceType=module&lineWrap=true&presets=env%2Creact%2Cstage-2%2Ctypescript&prettier=true&targets=&version=7.25.3&externalPlugins=&assumptions=%7B%7D)

## SSR（Server-side Rendering 服务端渲染）

服务端渲染是指页面的渲染过程发生在服务器。

与客户端渲染不同，在服务端渲染中浏览器接收到的 HTML 文档已经包含了完整的 DOM 结构，而非一个空的根 DOM。

服务端渲染过程如下：

1. **服务器获取数据**：当服务器接收到浏览器的页面请求后，获取页面渲染所需的数据。
2. **渲染 React 组件**：使用获取的数据渲染 React 组件。
3. **流式渲染**：调用 React Server 的 [`renderToPipeableStream`](https://react.dev/reference/react-dom/server/renderToPipeableStream) API，将 React 树以 HTML 形式渲染到 Node 流，并发送给浏览器。
4. **浏览器逐步渲染**：浏览器接收到流式响应后，逐步渲染页面，从而加快页面的首屏渲染速度。

调用过程如下：

```ts
import { renderToPipeableStream } from "react-dom/server";

const { pipe } = renderToPipeableStream(<App />, {
  bootstrapScripts: ["/main.js"],
  onShellReady() {
    response.setHeader("content-type", "text/html");
    pipe(response);
  },
});
```

但，此时浏览器渲染的页面是静态的，也就是仅具备 DOM 结构，但没有交互能力。

React 需要在前端对 HTML 进行水合（Hydration），将其转换为活跃页面，用户才能与其进行交互。

调用过程如下：

```ts
import { hydrateRoot } from "react-dom/client";

hydrateRoot(document.getElementById("root"), <App />);
```

> 水合是 SSR 特有的过程，在 CSR 中，由于页面内容完全由客户端通过 JS 生成，因此不需要水合页面即有交互能力。

### 优势

**首屏加载快**

服务端渲染

**SEO 友好**

SSR 生成的页面在服务器端就已经包含了完整的 HTML 内容，有利于搜索引擎抓取和索引，提高 SEO 效果。

### 不足

**不适合高动态页面**

对于需要频繁更新内容的动态页面，SSR 可能无法提供最佳体验，因为每次请求都需要重新生成页面内容。

**服务器负担重**

需要处理数据和页面渲染。

## SSG（Static Site Generation 静态站点生成）

静态站点生成是指页面的渲染过程发生在项目构建时，而不是在客户端或服务端的运行时。

在开发者运行构建命令时（如 Next.js 的 `next build`），构建工具会在服务器端或本地环境预先渲染所有页面，
根据每个页面生成一个静态 HTML 文件。这些 HTML 文件包含了预渲染的内容和样式，类似于 SSR 的结果，但这个渲染过程仅发生一次（即构建时）。

生成的静态 HTML 文件会被部署到静态文件服务器或 CDN 上。当用户访问网站时，服务器直接返回这些预生成的 HTML 文件，而无需在运行时进行任何额外的渲染操作。

静态生成（SSG）适用于博客等内容相对稳定、更新频率较低，且注重 SEO 的网站。

## ISR（Incremental Static Regeneration 增量静态再生）

增量静态再生（ISR）结合了静态站点生成（SSG）和服务端渲染（SSR）的特点。

在初次构建时，ISR 会像 SSG 一样预生成静态页面，并将其部署到服务器或 CDN。

当用户访问页面时，如果页面已经过期，ISR 会触发一个后台再生成过程。这个过程在服务器端完成，类似于 SSR，但与 SSR 不同的是，再生成的页面会被存储为静态文件，而不是每次请求都进行实时渲染。

具体来说，再生成的流程如下：

1. 用户请求过期的页面时，ISR 会启动一个后台再生成过程。
2. 服务器重新生成页面，并将新的 HTML 文件更新到缓存中。
3. 在生成过程完成后，下一个用户请求将会获得更新后的页面。

举个例子，假设一个 Next.js 页面设置了 `revalidate` 选项为 10 秒，这意味着页面在构建时将会生成静态 HTML 文件，并在 10 秒内保持不变。

具体流程如下：

1. **0-10 秒**：在初始构建完成后的前 10 秒内，用户请求的 HTML 文件都是之前生成的静态页面。页面不会重新生成，所有请求都直接返回缓存的静态页面。
2. **10 秒后**：当第一个请求到达并发现页面已经过期时（超过了 10 秒），ISR 会启动一个后台再生成过程。此时，服务器仍然会返回旧的缓存页面给用户，同时在后台生成新的页面。
3. **生成完成后**：当后台生成的页面完成后，下一个用户请求将会得到新的页面。新生成的页面会被存储为缓存静态文件，并在后续的 10 秒内继续使用，直到再次触发再生成过程。

这种机制保证了页面在大部分时间内快速响应，同时能够在后台静默地更新内容，确保用户能尽快看到最新的数据。


## 四种渲染方式汇总对比

| 渲染方式            | 渲染时机      | 优缺点                                                                                 |
| :------------------ | :------------ | :------------------------------------------------------------------------------------- |
| 客户端渲染（CSR）   | 运行时        | 支持动态内容和局部更新，交互更流畅。首屏加载较慢，且对搜索引擎的 SEO 支持有限。        |
| 服务端渲染（SSR）   | 运行时        | 首屏加载迅速，SEO 友好。服务器负担较重，需要处理数据和页面渲染，可能导致性能瓶颈。     |
| 静态站点生成（SSG） | 构建时        | 页面加载极快，服务器负担轻。但不能实时更新内容，交互性和动态功能受限。                 |
| 增量静态再生（ISR） | 构建时+运行时 | 结合了 SSG 和 SSR 的优点，页面加载极快，且支持在缓存过期后通过服务端渲染更新页面内容。 |
